In [1]:
import pandas as pd 
import numpy as np 

In [2]:
def main(day, hour, quarter, scen, random_seed):

    ### Get quarterly flow of a particular snapshot of the day
    edge_flow_df = pd.read_csv('outputs/edges_df/edges_df_scen{}_r{}_DY{}_HR{}_QT{}.csv'.format(scen, random_seed, day, hour, quarter))

    ### Get attributes and geometry of each edge
    network_attr_df = pd.read_csv('edges_residual_demand.csv')
    edge_flow_df = pd.merge(edge_flow_df, network_attr_df, on = ['edge_id_igraph'])
    edge_flow_df['edge_id_igraph_str'] = edge_flow_df['edge_id_igraph'].astype(str)
    edge_flow_df['voc'] = edge_flow_df['true_vol']*quarter/edge_flow_df['capacity']

    ### Merge results from two directions
    edge_flow_df['undir_uv_igraph'] = pd.DataFrame(np.sort(edge_flow_df[['start_igraph', 'end_igraph']].values, axis=1), columns=['small_igraph', 'large_igraph']).apply(lambda x:'%s_%s' % (x['small_igraph'],x['large_igraph']),axis=1)
    edge_flow_df_grp = edge_flow_df.groupby('undir_uv_igraph').agg({
            'true_vol': np.sum, 
            'tot_vol': np.sum, 
            'voc': np.max,
            'edge_id_igraph_str': lambda x: '-'.join(x),
            'geometry': 'first'}).reset_index()
    edge_flow_df_grp = edge_flow_df_grp.rename(columns={'true_vol': 'undirected_quart_vol', 'tot_vol': 'undirected_tot_vol', 'voc': 'larger_voc'})
    print(hour, quarter, np.max(edge_flow_df_grp['undirected_quart_vol']), np.max(edge_flow_df_grp['undirected_tot_vol']))

    edge_flow_df_grp.to_csv('outputs/quarterly_traffic/quarterly_traffic_scen{}_r{}_DY{}_HR{}_qt{}.csv'.format(scen, random_seed, day, hour, quarter), index=False)

In [3]:
day='na'
scen='base'
random_seed = 0
for hour in [0,1,2,3]:
    for quarter in [0,1,2,3]:
        main(day, hour, quarter, scen, random_seed)

0 0 2568.0 2568.0
0 1 2649.0 5217.0
0 2 2364.0 7581.0
0 3 2612.0 10193.0
1 0 1987.0 11021.0
1 1 1888.0 11746.0
1 2 1900.0 12113.0
1 3 2168.0 12664.0
2 0 1893.0 13096.0
2 1 1937.0 13492.0
2 2 1716.0 13914.0
2 3 1938.0 14424.0
3 0 1956.0 14577.0
3 1 1556.0 14595.0
3 2 1997.0 14597.0
3 3 26.0 14597.0
